# PETs/TETs – Hyperledger Aries / PySyft – Manufacturer 1 (Holder) 💼

In [1]:
%%javascript
document.title = '💼 Manufacturer1 Agent'

<IPython.core.display.Javascript object>

## PART 3: Connect with City to Analyze Data

**What:** -

**Why:** -

**How:** <br>


**Accompanying Agents and Notebooks:**
* City 🏙️️: `03_connect_with_manufacturer1.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [2]:
import libs.helpers as helpers
import os
import syft as sy
import time
import torch
import sympc

from aries_cloudcontroller import AriesAgentController
from libs.agent_connection_manager import CredentialHolder
from pprintpp import pprint
from termcolor import colored
from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor

#### 0.2 – Variables

In [3]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

---

<a id=1></a>

### 1 – Initiate Manufacturer1 Agent
#### 1.1 – Init ACA-PY agent controller

In [4]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Initialising a controller with admin api at http://manufacturer1-agent:3021 and an api key of adminApiKey


#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [5]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.3 – Init ACM issuing authority

In [6]:
# The CredentialHolder registers relevant webhook servers and event listeners
manufacturer1_agent = CredentialHolder(agent_controller)

# Verify if Manufacturer already has a VC
# (if there are manufacturer credentials, there is no need to execute the notebook)
manufacturer1_agent.get_credentials()

Successfully initiated AgentConnectionManager for a(n) Holder ACA-PY agent


{'results': []}

---

<a id=2></a>

### 2 – Establish a connection with the City agent 🏙️
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the manufacturer1 requests a connection with the Authority to be certified as an official city agency. Thus, the manufacturer1 agent sends an invitation to the Authority. In real life, the invitation can be shared via video call, phone call, or E-Mail. In this PoC, this is represented by copy and pasting the invitation into the manufacturers' notebooks.

#### 2.1 Join invitation of City agent 🏙️
Copy and paste the multi-use invitation of the city agent, and establish a connection with them.

In [7]:
# Variables
alias = "undisclosedM1"
auto_accept = True

# Receive connection invitation
connection_id = manufacturer1_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept)

Please enter invitation received by external agent:


Invitation:  {     '@id': '83c7c09f-a422-4983-9348-b1e123f1fe7d',     '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',     'label': 'City-Agency',     'recipientKeys': ['GjWMfWa7fNjSL5MLjPYBaPmmpTAZ8cMqQwpxyPxBg5Rz'],     'serviceEndpoint': 'https://09fc-193-9-114-179.ngrok.io', }



---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  bff477eb-11e6-4b0c-84b4-f890ac1e7733
State :  invitation (invitation-received)
Routing State : none
Connection with :  City-Agency
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  bff477eb-11e6-4b0c-84b4-f890ac1e7733
State :  request (request-sent)
Routing State : none
Connection with :  City-Agency
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  bff477eb-11e6-4b0c-84b4-f890ac1e7733
State :  response (response-received)
Routing State : none
Connection with :  City-Agency
Their Ro

<div style="font-size: 25px"><center><b>Break Point 2/3/4</b></center></div>
<div style="font-size: 50px"><center>💼 ➡️ 💼 / 💼 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers. <br> If you have established a connection between the City and all Manufacturers, proceed to the City Notebook's Step 2.3</b></center>

---

### 3 – Create Presentation to Send Proof Presentation

#### 3.1 – Create presentation that satisfies requirements of proof request
Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. To do so, the `prepare_presentation()` function runs through the following steps: 
1. Get all proof requests that were sent through `connection_id`
2. Get the most recent `presentation_exchange_id` and the corresponding `proof_request` from (1)
3. Get the restrictions the City agent defined in `proof_request` from (2)
4. Compare all VCs the Manufacturer1 agent has stored, and find (if available) a VC that satisfies the restrictions from (3)
5. Return a presentation dictionary from a VC from (4) that satisfies all requirements. Generally, a presentation consists of three classes of attributes: <br>
a. `requested_attributes`: Attributes that were signed by an issuer and have been revealed in the presentation process <br>
b. `self_attested_attributes`: Attributes that the prover has self attested to in the presentation object. <br>
c. `requested_predicates` (predicate proofs): Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

In [ ]:
presentation, presentation_exchange_id = manufacturer1_agent.prepare_presentation(connection_id)

#### 3.2 – Send Presentation

Send the presentation to the recipient of `presentation_exchange_id`

In [ ]:
manufacturer1_agent.send_proof_presentation(presentation_exchange_id, presentation)

<div style="font-size: 25px"><center><b>Break Point 6/7/8</b></center></div>
<div style="font-size: 50px"><center>💼 ➡️ 💼 / 💼 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers and run all cells between Steps 3 and 4.1 <br> If you have sent proof presentations from all manufacturers, proceed to the City Notebook's Step 3.3 </b></center>


---

### 4 – Do Data Science

#### 4.1 – Establish a Duet Connection: Send Duet invitation
Duet is a package that allows you to exchange encrypted data and run privacy-preserving arithmetic operations on them (e.g., through homomorphic encryption or secure multiparty computation).

In [8]:
# Set up connection_id to use for duet connection
manufacturer1_agent._update_connection(connection_id=connection_id, is_duet_connection=True, reset_duet=True)

# Create duet invitation for city agent
duet = sy.launch_duet(credential_exchanger=manufacturer1_agent)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Sending Duet Token 3c19d7f967d8e2b5f9e66a8eee3c3c0c
♫♫♫ > to Duet Partner City-Agency
♫♫♫ > via Connection ID bff477eb-11e6-4b0c-84b4-f890ac1e7733
♫♫♫ > Done!

♫♫♫ > STEP 2: Awaiting Duet Token from Duet Partner...

♫♫♫ > DONE! Partner's Duet Token: 1736525906a45a956af050987815ecac
♫♫♫ > Connecting...

♫♫♫ > CONNECTED!

10389 DUET LIVE STATUS  -  Objects: 1  Requests: 0   Messages: 9  Request Handlers: 0                                
275 10390
276 10391
277 10392
278 10393
279 10394
280 10395
281 10396
282 10397
283 10398
284 10399
♫♫

#### 4.2 - Load data to duet store

In [9]:
# Step 8: test there is no data in the server atm
duet.store.pandas

""


In [10]:
#age_data = torch.tensor([50,23,72,83])
#age_data = age_data.tag("test_data_age")
#age_data = age_data.describe("description for age_data")
#age_data_pointer = age_data.send(duet, pointable=True)

In [11]:
import pandas as pd
import numpy as np

In [27]:
# Get dataframe and get hour of every entry from timestamp (zipcode data from https://daten.odis-berlin.de/de/dataset/plz/)
df_zipcode = pd.read_csv("data/berlin_zipcodes.csv").rename(columns={"pls":"zipcode"})
df = pd.read_csv("data/data.csv", index_col=0)
df["hour"] = df.timestamp.apply(lambda x: int(x[11:13]))
df.head()

,i,vehicle_id,manufacturer_id,zipcode,timestamp,latlon,dist,seconds,co2_grams,total_dist,total_seconds,total_co2_grams,timestamp_tripstart,avg_kmperhour,avg_co2perkm,trip_id
0,0,V38924682,V88337,14195,2021-08-19 07:21:42,"(52.461766, 13.2364254)",0.00,0.0,0.00,0.00,0.0,0.0,2021-08-19 07:21:42,0.000000,0.000000,1
1,1,V38924682,V88337,14195,2021-08-19 07:21:54,"(52.4607628, 13.2387575)",0.19,12.0,17.48,0.19,12.0,17.0,2021-08-19 07:21:42,57.000000,89.473684,1
2,2,V38924682,V88337,14195,2021-08-19 07:21:59,"(52.4604334, 13.2393669)",0.06,5.0,5.22,0.25,17.0,22.0,2021-08-19 07:21:42,52.941176,88.000000,1
3,3,V38924682,V88337,14195,2021-08-19 07:22:13,"(52.4592834, 13.2412835)",0.18,14.0,15.84,0.43,31.0,38.0,2021-08-19 07:21:42,49.935484,88.372093,1
4,4,V38924682,V88337,14195,2021-08-19 07:22:29,"(52.4583943, 13.2432964)",0.17,16.0,13.94,0.60,47.0,52.0,2021-08-19 07:21:42,45.957447,86.666667,1


In [137]:
# Group entries by zipcode #and hour, and sum up the co2 emmitted
# Goal: give the city a CO2 emission per zipcode on August 19th
df_co2 = df.groupby(["zipcode"]).sum()[["co2_grams"]].reset_index()

# Berlin Zipcodes range between 10115 and 14199 (see https://en.wikipedia.org/wiki/List_of_postal_codes_in_Germany)
# Thus, we create a default tensor that ranges between 10115 (with index = 0) and 14199 (with index = 4084)
df_co2_per_zipcode = pd.merge(df_zipcodes, df_co2, on=["zipcode"], how="outer").replace(np.nan, 0)
print(df_co2_per_zipcode.shape)
df_co2_per_zipcode.head()

(4085, 2)


,zipcode,co2_grams
0,10115,0.0
1,10116,0.0
2,10117,0.0
3,10118,0.0
4,10119,0.0


In [75]:
#  Upload daily tensor
co2_per_zip = df_co2_per_zipcode["co2_grams"].tolist()
co2_torch = torch.tensor(co2_per_zip)
co2_torch = co2_torch.tag("co2-per-zip_2021-08-19")
co2_torch = co2_torch.describe("Total CO2 per Zipcode on August 19, 2021")
co2_torch_pointer = co2_torch.send(duet, pointable=True)

In [144]:
# Get hourly co2 
df_hourly_co2 = df[["zipcode", "hour","co2_grams"]].groupby(["zipcode", "hour"]).sum().reset_index()
df_hourly_co2 = df_hourly_co2.pivot(index=["zipcode"], columns=["hour"])["co2_grams"].replace(np.nan, 0)

# Get matrix that of shape (4085,25)
df_hourly_zipcode = df_zipcodes.set_index("zipcode").reindex(columns=list(range(0,24))).replace(np.nan,0)#.reset_index()

# Add matrix-shaped pandas
df_hourly_co2_per_zipcode = df_hourly_zipcode.add(df_hourly_co2, fill_value=0)
print(df_hourly_co2_per_zipcode.shape)
df_hourly_co2_per_zipcode.head()

(4085, 24)


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
zipcode,,,,,,,,,,,,,,,,,,,,,
10115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [146]:
hourly_co2_torch = torch.tensor(df_hourly_co2_per_zipcode.values)
hourly_co2_torch = hourly_co2_torch.tag("hourly-co2-per-zip_2021-08-19")
hourly_co2_torch = hourly_co2_torch.describe("Total CO2 per Zipcode per Hour on August 19, 2021. Shape: zipcode (10115-14199) x hour (0-23) = 4085 x 24")
hourly_co2_torch_pointer = hourly_co2_torch.send(duet, pointable=True)

In [147]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: fc8e7f182b53441397768ee006326736>,[],,<class 'sympc.session.session.Session'>
1,<UID: 12f6c5fa27794d99a1e9e935b1562b4e>,[co2-per-zip_2021-08-19],"Total CO2 per Zipcode on August 19, 2021",<class 'torch.Tensor'>
2,<UID: 3faf1f075cec4f35abee8c4688b19650>,[hourly-co2-per-zip_2021-08-19],"Total CO2 per Zipcode per Hour on August 19, 2...",<class 'torch.Tensor'>


In [78]:
# Step 11: Authorize the requests ds can make
duet.requests.add_handler(action="accept")

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()